In [ ]:
%cd /app

import os
import pandas as pd
import datetime
import numpy as np
from pandas import IndexSlice as idx

In [ ]:
exports_df = pd.read_excel("data/us_crude_exports.xls", sheet_name="Data 1", index_col=0)
imports_df = pd.read_excel("data/us_crude_import.xls", sheet_name="Data 1", index_col=0)
production_df = pd.read_excel("data/us_crude_production.xls", sheet_name="Data 1", index_col=0)
cushing_df = pd.read_excel("data/weekly_cushing_stocks.xls", sheet_name="Data 1", index_col=0)
total_stocks_df = pd.read_excel("data/weekly_petroleum_stocks.xls", sheet_name="Data 1", index_col=0)

rename_stocks = {
    "WCRSTUS1":"total_us_inc_spr",
    "WCSSTUS1":"spr"
}

total_stocks_df.rename(mapper=rename_stocks, axis=1, inplace=True)

balance_df = pd.concat([exports_df, imports_df, production_df, cushing_df, total_stocks_df], axis=1, join="inner")
balance_df['total_us_ex_spr'] = balance_df['total_us_inc_spr'] - balance_df['spr']
balance_df['net_exports'] = balance_df['crude_exports'] - balance_df['crude_imports']

In [ ]:
balance_4wk_df = balance_df.rolling(window=4).mean()

In [ ]:
for x in ['total_us_ex_spr', 'cushing_stocks']:
    balance_4wk_df[f"{x}_5yr_deviation"] = balance_4wk_df[x] - balance_df[x].rolling(window=52*5).mean()

In [ ]:
balance_yoy_df = balance_4wk_df - balance_4wk_df.shift(52)
balance_yoy_df.columns = [f"{x}_yoy" for x in balance_yoy_df.columns]



pd.concat([balance_4wk_df, balance_yoy_df], axis=1, join="inner").to_csv("data/balance_data.csv")

In [ ]:
#balance_yoy_df['cushing_stocks_yoy'].plot()